# Automated Essay Scoring

In [134]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices())

2.8.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [135]:
# import important libraries and download data
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score, r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import multiprocessing
%matplotlib notebook
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/SonTungKieu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/SonTungKieu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/SonTungKieu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [136]:
# Declaring some visualization methods to plot accuracy and model diagram
def plot_loss_curve(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['mae'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

def plot_acrchitecture(filename, model):
    plot_model(model, to_file=str(filename) + '.png')

In [137]:
# method to split data into sets
def split_in_sets(data):
    essay_sets = []
    min_scores = []
    max_scores = []
    
    for s in range(1,9):
        essay_set = data[data["essay_set"] == s]
        essay_set.dropna(axis=1, inplace=True)
        n, d = essay_set.shape
        set_scores = essay_set["domain1_score"]
        print ("Set", s, ": Essays = ", n , "\t Attributes = ", d)
        min_scores.append(set_scores.min())
        max_scores.append(set_scores.max())
        essay_sets.append(essay_set)
    
    return essay_sets, min_scores, max_scores

In the below cell, we can see the data we need to operate. We essentially drops the column, we dont need and keep the domain_score only along with essay text.

In [138]:
dataset_path = "./data/training_set_rel3.tsv"
data = pd.read_csv(dataset_path, sep="\t", encoding="ISO-8859-1")
data.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
data.shape

(12976, 28)

In [140]:
min_scores = [2, 1, 0, 0, 0, 0, 0, 0]
max_scores = [12, 6, 3, 3, 4, 4, 30, 60]
essay_sets, data_min_scores, data_max_scores = split_in_sets(data)
set1, set2, set3, set4, set5, set6, set7, set8 = tuple(essay_sets)
data.dropna(axis=1, inplace=True)
data.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set1.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set2.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set3.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set4.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set5.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set6.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set7.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set8.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
sets = [set1,set2,set3,set4,set5,set6,set7,set8]
data.head()

Set 1 : Essays =  1783 	 Attributes =  6
Set 2 : Essays =  1800 	 Attributes =  9
Set 3 : Essays =  1726 	 Attributes =  6
Set 4 : Essays =  1770 	 Attributes =  6
Set 5 : Essays =  1805 	 Attributes =  6
Set 6 : Essays =  1800 	 Attributes =  6
Set 7 : Essays =  1569 	 Attributes =  14
Set 8 : Essays =  723 	 Attributes =  18


,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [141]:
cap = ['@CAPS'+str(i) for i in range(100)]
loc = ['@LOCATION'+str(i) for i in range(100)]
org =['@ORGANIZATION'+str(i) for i in range(100)]
per = ['@PERSON'+str(i) for i in range(100)]
date = ['@DATE'+str(i) for i in range(100)]
time = ['@TIME'+str(i) for i in range(100)]
money = ['@MONEY'+str(i) for i in range(100)]
ner =  cap + loc + org + per + date + time + money

## Score distribution

In [142]:
topic_number = 0
fig, ax = plt.subplots(4,2, figsize=(15, 12), sharey=False)

for i in range(4):
    for j in range(2):
        topic_number += 1
        data[data['essay_set'] == topic_number].groupby('domain1_score')['essay_id'].agg('count').plot.bar(ax=ax[i, j], rot=0, color="maroon")
        ax[i,j].set_title('Essay %i' % topic_number)

# ax[3,0].locator_params(nbins=10)
# ax[3,1].locator_params(nbins=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

<IPython.core.display.Javascript object>

In [143]:
essay_sets, data_min_scores, data_max_scores = split_in_sets(data)
set1, set2, set3, set4, set5, set6, set7, set8 = tuple(essay_sets)
data.dropna(axis=1, inplace=True)
sets = [set1,set2,set3,set4,set5,set6,set7,set8]
set1.head()

Set 1 : Essays =  1783 	 Attributes =  4
Set 2 : Essays =  1800 	 Attributes =  4
Set 3 : Essays =  1726 	 Attributes =  4
Set 4 : Essays =  1770 	 Attributes =  4
Set 5 : Essays =  1805 	 Attributes =  4
Set 6 : Essays =  1800 	 Attributes =  4
Set 7 : Essays =  1569 	 Attributes =  4
Set 8 : Essays =  723 	 Attributes =  4


,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


# Model

In [144]:
from tensorflow.keras.layers import Embedding, Input, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional, Conv2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Sequential,Model, load_model, model_from_config
import keras.backend as K
from tensorflow.keras.optimizers import Adam

In [145]:
def get_model(Hidden_dim1=400, Hidden_dim2=128, return_sequences = True, dropout=0.3, recurrent_dropout=0.3, input_size_x=3, input_size_y=768, activation='relu', bidirectional = False):
    """Define the model."""
    model = Sequential()
    if bidirectional:
        model.add(Bidirectional(LSTM(Hidden_dim1,return_sequences=return_sequences , dropout=0.4, recurrent_dropout=recurrent_dropout), input_shape=[input_size_x, input_size_y]))
        model.add(Bidirectional(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout, return_sequences=return_sequences)))
        model.add(Bidirectional(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout)))
    else:
        model.add(LSTM(Hidden_dim1, dropout=0.4, recurrent_dropout=recurrent_dropout, input_shape=[input_size_x, input_size_y], return_sequences=return_sequences))
        model.add(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout, return_sequences=return_sequences))
        model.add(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout))

    model.add(Dropout(dropout))
    model.add(Dense(128, activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dense(32, activation=activation))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae', tf.keras.losses.MeanAbsolutePercentageError()])
    model.summary()
    
    return model

In [146]:
def get_features(X):
    max_len = 0
    for i in X.values:
        if len(i) > max_len:
            max_len = len(i)

    padded_X = np.array([i + [0]*(max_len-len(i)) for i in X.values])
    attention_mask_X = np.where(padded_X != 0, 1, 0)
    X_input_ids = torch.tensor(padded_X)
    X_attention_mask = torch.tensor(attention_mask_X)

    with torch.no_grad():
        last_hidden_states_X = model(X_input_ids, attention_mask=X_attention_mask)

    X_features = last_hidden_states_X[0][:,0,:].numpy()

    return X_features


In [151]:
def convert_to_tensors(input_ids, chunksize = 512):

    input_id_chunks = list(input_ids)

    if len(input_id_chunks) > 1:
        for i, j in enumerate(input_id_chunks):
            input_id_chunks[i] = torch.cat([torch.Tensor([101]), j, torch.Tensor([102])])
            
            pad_len = chunksize - input_id_chunks[i].shape[0]
                
            if pad_len > 0:
                input_id_chunks[i] = torch.cat([input_id_chunks[i], torch.Tensor([0] * pad_len)])
            
            # print(input_id_chunks[i])

        input_ids_tensor = torch.stack(input_id_chunks)

    else:
        input_id_chunks = torch.cat([torch.Tensor([101]), input_id_chunks[0], torch.Tensor([102])])
        
        pad_len = chunksize - input_id_chunks.shape[0]
        
        if pad_len > 0:
            input_id_chunks = torch.cat([input_id_chunks, torch.Tensor([0] * pad_len)])

        input_ids_tensor = input_id_chunks.view(1, chunksize)

    return input_ids_tensor
        

In [148]:
def get_longest_tensor(X_train, X_test):
    max_length = 0
    
    for tensor in X_train:
        if len(tensor) > max_length:
            max_length = len(tensor)
    
    for tensor in X_test:
        if len(tensor) > max_length:
            max_length = len(tensor)
    
    return max_length

In [149]:
# Add tensor([0] * 512) to tensors that length < longest_tensor
def add_padding_tensors(x, longest_tensor, chunksize=512):
    if len(x) < longest_tensor:
        no_padding_left = longest_tensor - len(x)
        
        zero_padding = torch.Tensor([0] * chunksize)
        zero_paddings = zero_padding.repeat(no_padding_left, 1)
        final_tensor = torch.cat((x, zero_paddings), 0)
    
    else:
        final_tensor = x

    return final_tensor


In [150]:
# Create new directory to store model weights
import shutil

path = './saved_model'

if os.path.isdir(path):
    shutil.rmtree(path)
    print('Removed!')
    os.mkdir(path)
    print('Done!')
else:
    os.mkdir(path)
    print('Done!')

Removed!
Done!


In [152]:
# Chay thu model o day nha em yeu

import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
set_count = 1
all_sets_score = []

# Metrics
mape = tf.keras.losses.MeanAbsolutePercentageError()
mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()

batch_size = 50
epochs = 70


for s in sets:
    print("\n--------SET {}--------\n".format(set_count))
    X = s
    y = s['domain1_score']
    cv = KFold(n_splits=3, shuffle=True)
    cv_data = cv.split(X)

    # Create new directory to store model weights of each set
    os.mkdir('./saved_model/set{}'.format(set_count))
    saved_model_set_path = './saved_model/set{}'.format(set_count)

    kappa_results = []
    mse_results = []
    mae_results = []
    mape_results = []

    prediction_list = []
    fold_count = 1
#     cuda = torch.device('cuda')
    model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel,
                                                        ppb.DistilBertTokenizer,
                                                        'distilbert-base-uncased')
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

    for traincv, testcv in cv_data:
        print("\n--------Fold {}--------\n".format(fold_count))

        checkpoint_path = saved_model_set_path + '/cp{}.h5'.format(fold_count)

        # get the train and test from the dataset.
        X_train, X_test, y_train, y_test = X.iloc[traincv], X.iloc[testcv], y.iloc[traincv], y.iloc[testcv]
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

        train_essays = X_train['essay']
        val_essays = X_val['essay']
        test_essays = X_test['essay']

        tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x,
                                                                         add_special_tokens=False,
                                                                         return_tensors='pt')))
        tokenized_val = val_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=False, return_tensors='pt')))
        tokenized_test = test_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=False, return_tensors='pt')))

        tokenized_train = tokenized_train.apply(lambda x: x[0].split(510) if len(x[0]) > 510 else x)
        tokenized_val = tokenized_val.apply(lambda x: x[0].split(510) if len(x[0]) > 510 else x)
        tokenized_test = tokenized_test.apply(lambda x: x[0].split(510) if len(x[0]) > 510 else x)
        
        tokenized_train = tokenized_train.reset_index()['essay']
        tokenized_val = tokenized_val.reset_index()['essay']
        tokenized_test = tokenized_test.reset_index()['essay']

        tokenized_train = tokenized_train.apply(lambda x: convert_to_tensors(x))
        tokenized_val = tokenized_val.apply(lambda x: convert_to_tensors(x))
        tokenized_test = tokenized_test.apply(lambda x: convert_to_tensors(x))

        longest_tensor = get_longest_tensor(tokenized_train, tokenized_test)

        # Add padding to tensors
        tokenized_train = tokenized_train.apply(lambda x: add_padding_tensors(x, longest_tensor))
        tokenized_val = tokenized_val.apply(lambda x: add_padding_tensors(x, longest_tensor))
        tokenized_test = tokenized_test.apply(lambda x: add_padding_tensors(x, longest_tensor))

        tokenized_train = tokenized_train.to_numpy()
        tokenized_train = np.array([np.array(val) for val in tokenized_train])

        tokenized_val = tokenized_val.to_numpy()
        tokenized_val = np.array([np.array(val) for val in tokenized_val])

        tokenized_test = tokenized_test.to_numpy()
        tokenized_test = np.array([np.array(val) for val in tokenized_test])

        # Checkpoint
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         verbose=1,
                                                         save_weights_only=True,
                                                         save_best_only=True)

         # Bidirectional LSTM Model
        bi_dir_lstm_model = get_model(bidirectional=True, input_size_x=tokenized_train.shape[1], input_size_y=tokenized_train.shape[2])
        history = bi_dir_lstm_model.fit(tokenized_train, y_train, batch_size=batch_size, epochs=epochs,
                                        validation_data=(tokenized_val, y_val),
                                        callbacks=[cp_callback])
        # history.save_weights('./saved_model/set{}/model_set{}_fold{}'.format(set_count, set_count, fold_count))


        # Plot loss
        plot_loss_curve(history)

        y_pred = bi_dir_lstm_model.predict(tokenized_test)
        y_pred = np.around(y_pred)
        np.nan_to_num(y_pred)

        y_test = y_test.to_numpy().reshape(-1, 1)

        y_pred = y_pred.reshape(-1, 1)

        result_tbl = pd.DataFrame(data=np.concatenate((y_test, y_pred), axis=1),
                                  columns=['Test', 'Pred'])
        print(result_tbl)

        # evaluate the model

        kappa = cohen_kappa_score(y_test, y_pred, weights='quadratic')
        mape_score = mape(y_test, y_pred).numpy()
        mse_score = mse(y_test, y_pred).numpy()
        mae_score = mae(y_test, y_pred).numpy()

        print("Kappa Score: {}".format(kappa))
        print('MSE {}'.format(mse_score))
        print('MAE {}'.format(mae_score))
        print('MAPE {}'.format(mape_score))

        kappa_results.append(kappa)
        mse_results.append(mse_score)
        mae_results.append(mae_score)
        mape_results.append(mape_score)

        fold_count +=1

        tf.keras.backend.clear_session()

        all_sets_score.append(kappa_results)

    print("Average Kappa score value is : {}".format(np.mean(np.asarray(kappa_results))))
    print("Average MSE score value is : {}".format(np.mean(np.asarray(mse_results))))
    print("Average MAE score value is : {}".format(np.mean(np.asarray(mae_results))))
    print("Average MAPE score value is : {}".format(np.mean(np.asarray(mape_results))))
    set_count+=1


--------SET 1--------



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (536 > 512). Running this sequence through the model will result in indexing errors



--------Fold 1--------

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (None, 3, 800)           2921600   
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 3, 256)           951296    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                             

2022-06-06 23:52:32.225854: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - ETA: 0s - loss: 25.0809 - mse: 25.0809 - mae: 4.0487 - mean_absolute_percentage_error: 48.4737

...

8/8 [==============================] - ETA: 0s - loss: 1348.1901 - mse: 1348.1901 - mae: 36.2388 - mean_absolute_percentage_error: 97.8369

2022-06-07 01:42:03.237159: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 1024.18689, saving model to ./saved_model/set8/cp3.h5
8/8 [==============================] - 21s 1s/step - loss: 1348.1901 - mse: 1348.1901 - mae: 36.2388 - mean_absolute_percentage_error: 97.8369 - val_loss: 1024.1869 - val_mse: 1024.1869 - val_mae: 31.6419 - val_mean_absolute_percentage_error: 87.7164
Epoch 2/70
8/8 [==============================] - ETA: 0s - loss: 688.1785 - mse: 688.1785 - mae: 24.9264 - mean_absolute_percentage_error: 65.6081
Epoch 2: val_loss improved from 1024.18689 to 103.62378, saving model to ./saved_model/set8/cp3.h5
8/8 [==============================] - 4s 487ms/step - loss: 688.1785 - mse: 688.1785 - mae: 24.9264 - mean_absolute_percentage_error: 65.6081 - val_loss: 103.6238 - val_mse: 103.6238 - val_mae: 8.9734 - val_mean_absolute_percentage_error: 23.5717
Epoch 3/70
8/8 [==============================] - ETA: 0s - loss: 97.2909 - mse: 97.2909 - mae: 7.9415 - mean_absolute_percentage_error: 23.4685
Epoch 3: val_lo

2022-06-07 01:45:03.119390: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


     Test  Pred
0    46.0  39.0
1    40.0  39.0
2    40.0  38.0
3    20.0  28.0
4    30.0  38.0
..    ...   ...
236  40.0  38.0
237  40.0  39.0
238  10.0  30.0
239  32.0  37.0
240  40.0  33.0

[241 rows x 2 columns]
Kappa Score: 0.4959136384512203
MSE 22.954357147216797
MAE 3.7095437049865723
MAPE 10.757608413696289
Average Kappa score value is : 0.4373673703579555
Average MSE score value is : 24.568466186523438
Average MAE score value is : 3.868603467941284
Average MAPE score value is : 10.913330078125


In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# x = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
def plot_qwk_scores_all_sets():
  fig = plt.figure()
  ax = plt.subplot(111)
  x = [1,2,3,4,5]
  ax.plot(x, set1 , label='set1')
  ax.plot(x, set2, label='set2')
  ax.plot(x, set3, label='set3')
  ax.plot(x, set4, label='set4')
  ax.plot(x, set5, label='set5')
  ax.plot(x, set6, label='set6')
  ax.plot(x, set7, label='set7')
  ax.plot(x, set8, label='set8')
  plt.title('Set wise QWK using BERT for individual sets')
  ax.legend()
  plt.show()
